[知乎专栏](https://zhuanlan.zhihu.com/p/25742261)

# General Appproach


## 1. Data Exploration

EDA(Exploratory Data Analysis)对数据进行探索性的分析，通常会使用pandas来载入数据

### 1.1 Visualization

使用matplotlib 和 seaborn

常用的图表：、
- 查看目标变量的分布。当分布不平衡时，根据评分标准和具体模型的使用不同，可能会严重影响性能。
- 对 **Numerical Variable**，可以用 **Box Plot** 来直观地查看它的分布。
- 对于**坐标类数据**，可以用 **Scatter Plot** 来查看它们的分布趋势和是否有离群点的存在。
- 对于**分类问题**，将数据根据 Label 的不同着**不同的颜色**绘制出来，这对 Feature 的构造很有帮助。
- 绘制变量之间两两的分布和相关度图表。
iris数据集可视化例子 https://www.kaggle.com/benhamner/python-data-visualizations

### 1.2 Statistical  Tests
对数据进行一些统计上的测试来验证一些假设的显著性。

## 2. Data Preprocessing

- 有时数据会分散在几个不同的文件中，需要join
- 处理Missing Data
- 处理outlier
- 必要时转换某些Categorical Variable
- 有些 Float 变量可能是从未知的 Int 变量转换得到的，这个过程中发生精度损失会在数据中产生不必要的 Noise，即两个数值原本是相同的却在小数点后某一位开始有不同。这对 Model 可能会产生很负面的影响，需要设法去除或者减弱 Noise。

outlier: 去除数据中的一些离群点。
Dummy Variables: 虚拟变量，对于categorical variable ，使用ont-hot-encoding


## 3. Feature Engineering
> kaggle比赛：**Feature 为主，调参和Ensemble(集成)为辅**

> 当一个变量从直觉上来说对所要完成的目标有帮助，就可以作为Feature

### 3.1 Feature Selection
> 我们应该生成尽量多的 Feature，相信 Model 能够挑出最有用的 Feature

- Feature 越少，训练越快。
- 有些 Feature 之间可能存在线性关系，影响 Model 的性能。
- **通过挑选出最重要的 Feature，可以将它们之间进行各种运算和操作的结果作为新的 Feature，可能带来意外的提高。**

Feature Selection 最实用的方法也就是看 Random Forest 训练完以后得到的 Feature Importance 了。
从原理上来讲，增加 Random Forest 中树的数量可以在一定程度上加强其对于 Noisy Data 的 Robustness。

### 3.2 Feature Encoding
有些raw feature 需要经过一些转换才能起到较好的效果

假设有一个 Categorical Variable 一共有几万个取值可能，那么创建 Dummy Variables 的方法就不可行了。这时一个比较好的方法是根据 Feature Importance 或是这些取值本身在数据中的出现频率，为最重要（比如说前 95% 的 Importance）那些取值（有很大可能只有几个或是十几个）创建 Dummy Variables，而所有其他取值都归到一个“其他”类里面。

## 4. Model Selection

kaggle上最常用的模型基本都是基于树的模型：
- Gradient Boosting
- Random Forest
- Extra Randomized Trees
 以下模型往往在性能上稍逊一筹，但是很适合作为 Ensemble 的 Base Model
- SVM
- Linear Regression
- Logistic Regression
- Neural Networks

**Gradient Boosting 本身优秀的性能加上 Xgboost 高效的实现，使得它在 Kaggle 上广为使用**
https://dnc1994.com/2016/03/installing-xgboost-on-windows/

### 4.1 Model Training
![model-training1](../photo/model-training1.png)
![model-training2](../photo/model-training2.png)

### 4.2 Cross Validation
它让你知道你的 Model 有没有 Overfit，是不是真的能够 Generalize 到测试集上
*Public LB（验证集） 和 Private LB（测试集）*

## 5. Ensemble Generation
Ensemble Learning 是指将多个不同的 Base Model 组合成一个 Ensemble Model 的方法。它可以同时降低最终模型的 Bias 和 Variance（https://link.springer.com/chapter/10.1007%2F3-540-33019-4_19）


常见的 Ensemble 方法有这么几种：

- Bagging：使用训练数据的不同随机子集来训练每个 Base Model，最后进行每个 Base Model 权重相同的 Vote。也即 Random Forest 的原理。
- Boosting：迭代地训练 Base Model，每次根据上一个迭代中预测错误的情况修改训练样本的权重。也即 Gradient - Boosting 的原理。比 Bagging 效果好，但更容易 Overfit。
- Blending：用不相交的数据训练不同的 Base Model，将它们的输出取（加权）平均。实现简单，但对训练数据利用少了。
- Stacking：接下来会详细介绍。

从理论上讲，Ensemble 要成功，有两个要素：

- Base Model 之间的相关性要尽可能的小。这就是为什么非 Tree-based Model 往往表现不是最好但还是要将它们包括在 Ensemble 里面的原因。Ensemble 的 Diversity 越大，最终 Model 的 Bias 就越低。
- Base Model 之间的性能表现不能差距太大。这其实是一个 Trade-off，在实际中很有可能表现相近的 Model 只有寥寥几个而且它们之间相关性还不低。但是实践告诉我们即使在这种情况下 Ensemble 还是能大幅提高成绩。

### 5.1 Stacking

相比 Blending，Stacking 能更好地利用训练数据。以 5-Fold Stacking 为例，它的基本原理如图所示：
![5-flod](../photo/5-flod.jpg)
整个过程很像 Cross Validation。首先将训练数据分为 5 份，接下来一共 5 个迭代，每次迭代时，将 4 份数据作为 Training Set 对每个 Base Model 进行训练，然后在剩下一份 Hold-out Set 上进行预测。同时也要将其在测试数据上的预测保存下来。这样，每个 Base Model 在每次迭代时会对训练数据的其中 1 份做出预测，对测试数据的全部做出预测。5 个迭代都完成以后我们就获得了一个 #训练数据行数 x #Base Model 数量 的矩阵，这个矩阵接下来就作为第二层的 Model 的训练数据。当第二层的 Model 训练完以后，将之前保存的 Base Model 对测试数据的预测（因为每个 Base Model 被训练了 5 次，对测试数据的全体做了 5 次预测，所以对这 5 次求一个平均值，从而得到一个形状与第二层训练数据相同的矩阵）拿出来让它进行预测，就得到最后的输出。


![code-flod](../photo/code-flod.png)

## 6. Pipeline

https://github.com/ChenglongChen/Kaggle_CrowdFlower

- 模块化 Feature Transform，只需写很少的代码就能将新的 Feature 更新到训练集中。
- 自动化 Grid Search，只要预先设定好使用的 Model 和参数的候选，就能自动搜索并记录最佳的 Model。
- 自动化 Ensemble Generation，每个一段时间将现有最好的 K 个 Model 拿来做 Ensemble。
